In [1]:
import lmdb
import caffe
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import cv2
from caffe.proto import caffe_pb2

In [2]:
data = [img for img in glob.glob("../../dataset/AND_dataset/AND_dataset/AND_images/*png")]

In [3]:
links = pd.DataFrame(data)

In [4]:
links.columns = ['URL']

In [5]:
data = links.URL.str.split("[/]",expand=True)

In [6]:
data = data.add_prefix('f_')
data = data.drop(["f_0","f_1","f_2","f_3","f_4","f_5"],axis=1)
data.head()

,f_6
0,0320b_num4.png
1,0809b_num3.png
2,0116c_num4.png
3,0128b_num1.png
4,0634c_num1.png


In [7]:
def genPositiveData(data):
    writers = np.unique(list(data.f_6.str.split("[a-z]",1,expand=True)[0]))
    skeleton = pd.DataFrame()
    num = len(writers)
    for i,writer in enumerate(writers):
        query = str("f_6.str.contains('"+str(writer)+"')")
        temp = data.query(query)
        data1 = temp.sample(frac=1)
        data1 = data1.add_prefix('w1_')
        data2 = temp.sample(frac=1)
        data2 = data2.add_prefix('w2_')
        test = pd.concat([data1.reset_index(drop=True),data2.reset_index(drop=True)],axis=1)
        test = test.query("w1_f_6 != w2_f_6")
        skeleton = skeleton.append(test)
        #if(i %200): print("Percent Done: "+str(i/num))
    skeleton["sameWriter"] = 1
    return skeleton

def genNegativeData(data):
    data1 = data.sample(frac=1)
    data1 = data1.add_prefix('w1_')
    data2 = data.sample(frac=1)
    data2 = data2.add_prefix('w2_')
    negSamples = pd.concat([data1.reset_index(drop=True),data2.reset_index(drop=True)],axis=1)
    negSamples["sameWriter"] = 0
    return negSamples

In [8]:
posData = genPositiveData(data)

In [9]:
negData = genNegativeData(data)

In [10]:
data = pd.concat([posData,negData]).reset_index(drop=True)
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,w1_f_6,w2_f_6,sameWriter
0,0529b_num4.png,1429c_num4.png,0
1,1119a_num6.png,1185b_num2.png,0
2,0987c_num3.png,0638c_num1.png,0
3,1294b_num3.png,1294c_num2.png,1
4,0191a_num2.png,0191a_num1.png,1


In [220]:
train_images, test_images, train_label, test_label = train_test_split(data.iloc[:,0:2],data.iloc[:,2:3], test_size=0.20, random_state=666)

In [225]:
train_images = train_images.reset_index(drop=True)
test_images = test_images.reset_index(drop=True)
train_label = train_label.reset_index(drop=True)
test_label = test_label.reset_index(drop=True)

In [267]:
train_lmdb = './train_lmdb'
test_lmdb = './test_lmdb'

In [269]:
height,width = (64,64)

In [248]:
def make_datum(img1,img2, label):
    #image is numpy.ndarray format. BGR instead of RGB
    return caffe_pb2.Datum(
        channels=2,
        width=width,
        height=height,
        label=label,
        data=np.array([img1,img2]).tostring())

In [261]:
in_db = lmdb.open(train_lmdb, map_size=int(1e12))
path = "../../dataset/AND_dataset/AND_dataset/AND_images/"
with in_db.begin(write=True) as in_txn:
    for rows in range(len(train_images)):
        path1 = train_images["w1_f_6"][rows]
        path2 = train_images["w2_f_6"][rows]
        label = train_label["sameWriter"][rows]
        img1 = cv2.imread(str(path+path1), cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(str(path+path2), cv2.IMREAD_GRAYSCALE)
        datum = make_datum(img1,img2,label)
        in_txn.put('{:0>5d}'.format(rows).encode(), datum.SerializeToString())
        print('{:0>5d}'.format(rows) + ':' + img_path)
in_db.close()

00000:w2_f_6
00001:w2_f_6
00002:w2_f_6
00003:w2_f_6
00004:w2_f_6
00005:w2_f_6
00006:w2_f_6
00007:w2_f_6
00008:w2_f_6
00009:w2_f_6
00010:w2_f_6
00011:w2_f_6
00012:w2_f_6
00013:w2_f_6
00014:w2_f_6
00015:w2_f_6
00016:w2_f_6
00017:w2_f_6
00018:w2_f_6
00019:w2_f_6
00020:w2_f_6
00021:w2_f_6
00022:w2_f_6
00023:w2_f_6
00024:w2_f_6
00025:w2_f_6
00026:w2_f_6
00027:w2_f_6
00028:w2_f_6
00029:w2_f_6
00030:w2_f_6
00031:w2_f_6
00032:w2_f_6
00033:w2_f_6
00034:w2_f_6
00035:w2_f_6
00036:w2_f_6
00037:w2_f_6
00038:w2_f_6
00039:w2_f_6
00040:w2_f_6
00041:w2_f_6
00042:w2_f_6
00043:w2_f_6
00044:w2_f_6
00045:w2_f_6
00046:w2_f_6
00047:w2_f_6
00048:w2_f_6
00049:w2_f_6
00050:w2_f_6
00051:w2_f_6
00052:w2_f_6
00053:w2_f_6
00054:w2_f_6
00055:w2_f_6
00056:w2_f_6
00057:w2_f_6
00058:w2_f_6
00059:w2_f_6
00060:w2_f_6
00061:w2_f_6
00062:w2_f_6
00063:w2_f_6
00064:w2_f_6
00065:w2_f_6
00066:w2_f_6
00067:w2_f_6
00068:w2_f_6
00069:w2_f_6
00070:w2_f_6
00071:w2_f_6
00072:w2_f_6
00073:w2_f_6
00074:w2_f_6
00075:w2_f_6
00076:w2_f_6

00651:w2_f_6
00652:w2_f_6
00653:w2_f_6
00654:w2_f_6
00655:w2_f_6
00656:w2_f_6
00657:w2_f_6
00658:w2_f_6
00659:w2_f_6
00660:w2_f_6
00661:w2_f_6
00662:w2_f_6
00663:w2_f_6
00664:w2_f_6
00665:w2_f_6
00666:w2_f_6
00667:w2_f_6
00668:w2_f_6
00669:w2_f_6
00670:w2_f_6
00671:w2_f_6
00672:w2_f_6
00673:w2_f_6
00674:w2_f_6
00675:w2_f_6
00676:w2_f_6
00677:w2_f_6
00678:w2_f_6
00679:w2_f_6
00680:w2_f_6
00681:w2_f_6
00682:w2_f_6
00683:w2_f_6
00684:w2_f_6
00685:w2_f_6
00686:w2_f_6
00687:w2_f_6
00688:w2_f_6
00689:w2_f_6
00690:w2_f_6
00691:w2_f_6
00692:w2_f_6
00693:w2_f_6
00694:w2_f_6
00695:w2_f_6
00696:w2_f_6
00697:w2_f_6
00698:w2_f_6
00699:w2_f_6
00700:w2_f_6
00701:w2_f_6
00702:w2_f_6
00703:w2_f_6
00704:w2_f_6
00705:w2_f_6
00706:w2_f_6
00707:w2_f_6
00708:w2_f_6
00709:w2_f_6
00710:w2_f_6
00711:w2_f_6
00712:w2_f_6
00713:w2_f_6
00714:w2_f_6
00715:w2_f_6
00716:w2_f_6
00717:w2_f_6
00718:w2_f_6
00719:w2_f_6
00720:w2_f_6
00721:w2_f_6
00722:w2_f_6
00723:w2_f_6
00724:w2_f_6
00725:w2_f_6
00726:w2_f_6
00727:w2_f_6

01294:w2_f_6
01295:w2_f_6
01296:w2_f_6
01297:w2_f_6
01298:w2_f_6
01299:w2_f_6
01300:w2_f_6
01301:w2_f_6
01302:w2_f_6
01303:w2_f_6
01304:w2_f_6
01305:w2_f_6
01306:w2_f_6
01307:w2_f_6
01308:w2_f_6
01309:w2_f_6
01310:w2_f_6
01311:w2_f_6
01312:w2_f_6
01313:w2_f_6
01314:w2_f_6
01315:w2_f_6
01316:w2_f_6
01317:w2_f_6
01318:w2_f_6
01319:w2_f_6
01320:w2_f_6
01321:w2_f_6
01322:w2_f_6
01323:w2_f_6
01324:w2_f_6
01325:w2_f_6
01326:w2_f_6
01327:w2_f_6
01328:w2_f_6
01329:w2_f_6
01330:w2_f_6
01331:w2_f_6
01332:w2_f_6
01333:w2_f_6
01334:w2_f_6
01335:w2_f_6
01336:w2_f_6
01337:w2_f_6
01338:w2_f_6
01339:w2_f_6
01340:w2_f_6
01341:w2_f_6
01342:w2_f_6
01343:w2_f_6
01344:w2_f_6
01345:w2_f_6
01346:w2_f_6
01347:w2_f_6
01348:w2_f_6
01349:w2_f_6
01350:w2_f_6
01351:w2_f_6
01352:w2_f_6
01353:w2_f_6
01354:w2_f_6
01355:w2_f_6
01356:w2_f_6
01357:w2_f_6
01358:w2_f_6
01359:w2_f_6
01360:w2_f_6
01361:w2_f_6
01362:w2_f_6
01363:w2_f_6
01364:w2_f_6
01365:w2_f_6
01366:w2_f_6
01367:w2_f_6
01368:w2_f_6
01369:w2_f_6
01370:w2_f_6

01958:w2_f_6
01959:w2_f_6
01960:w2_f_6
01961:w2_f_6
01962:w2_f_6
01963:w2_f_6
01964:w2_f_6
01965:w2_f_6
01966:w2_f_6
01967:w2_f_6
01968:w2_f_6
01969:w2_f_6
01970:w2_f_6
01971:w2_f_6
01972:w2_f_6
01973:w2_f_6
01974:w2_f_6
01975:w2_f_6
01976:w2_f_6
01977:w2_f_6
01978:w2_f_6
01979:w2_f_6
01980:w2_f_6
01981:w2_f_6
01982:w2_f_6
01983:w2_f_6
01984:w2_f_6
01985:w2_f_6
01986:w2_f_6
01987:w2_f_6
01988:w2_f_6
01989:w2_f_6
01990:w2_f_6
01991:w2_f_6
01992:w2_f_6
01993:w2_f_6
01994:w2_f_6
01995:w2_f_6
01996:w2_f_6
01997:w2_f_6
01998:w2_f_6
01999:w2_f_6
02000:w2_f_6
02001:w2_f_6
02002:w2_f_6
02003:w2_f_6
02004:w2_f_6
02005:w2_f_6
02006:w2_f_6
02007:w2_f_6
02008:w2_f_6
02009:w2_f_6
02010:w2_f_6
02011:w2_f_6
02012:w2_f_6
02013:w2_f_6
02014:w2_f_6
02015:w2_f_6
02016:w2_f_6
02017:w2_f_6
02018:w2_f_6
02019:w2_f_6
02020:w2_f_6
02021:w2_f_6
02022:w2_f_6
02023:w2_f_6
02024:w2_f_6
02025:w2_f_6
02026:w2_f_6
02027:w2_f_6
02028:w2_f_6
02029:w2_f_6
02030:w2_f_6
02031:w2_f_6
02032:w2_f_6
02033:w2_f_6
02034:w2_f_6

02591:w2_f_6
02592:w2_f_6
02593:w2_f_6
02594:w2_f_6
02595:w2_f_6
02596:w2_f_6
02597:w2_f_6
02598:w2_f_6
02599:w2_f_6
02600:w2_f_6
02601:w2_f_6
02602:w2_f_6
02603:w2_f_6
02604:w2_f_6
02605:w2_f_6
02606:w2_f_6
02607:w2_f_6
02608:w2_f_6
02609:w2_f_6
02610:w2_f_6
02611:w2_f_6
02612:w2_f_6
02613:w2_f_6
02614:w2_f_6
02615:w2_f_6
02616:w2_f_6
02617:w2_f_6
02618:w2_f_6
02619:w2_f_6
02620:w2_f_6
02621:w2_f_6
02622:w2_f_6
02623:w2_f_6
02624:w2_f_6
02625:w2_f_6
02626:w2_f_6
02627:w2_f_6
02628:w2_f_6
02629:w2_f_6
02630:w2_f_6
02631:w2_f_6
02632:w2_f_6
02633:w2_f_6
02634:w2_f_6
02635:w2_f_6
02636:w2_f_6
02637:w2_f_6
02638:w2_f_6
02639:w2_f_6
02640:w2_f_6
02641:w2_f_6
02642:w2_f_6
02643:w2_f_6
02644:w2_f_6
02645:w2_f_6
02646:w2_f_6
02647:w2_f_6
02648:w2_f_6
02649:w2_f_6
02650:w2_f_6
02651:w2_f_6
02652:w2_f_6
02653:w2_f_6
02654:w2_f_6
02655:w2_f_6
02656:w2_f_6
02657:w2_f_6
02658:w2_f_6
02659:w2_f_6
02660:w2_f_6
02661:w2_f_6
02662:w2_f_6
02663:w2_f_6
02664:w2_f_6
02665:w2_f_6
02666:w2_f_6
02667:w2_f_6

...

17983:w2_f_6
17984:w2_f_6
17985:w2_f_6
17986:w2_f_6
17987:w2_f_6
17988:w2_f_6
17989:w2_f_6
17990:w2_f_6
17991:w2_f_6
17992:w2_f_6
17993:w2_f_6
17994:w2_f_6
17995:w2_f_6
17996:w2_f_6
17997:w2_f_6
17998:w2_f_6
17999:w2_f_6
18000:w2_f_6
18001:w2_f_6
18002:w2_f_6
18003:w2_f_6
18004:w2_f_6
18005:w2_f_6
18006:w2_f_6
18007:w2_f_6
18008:w2_f_6
18009:w2_f_6
18010:w2_f_6
18011:w2_f_6
18012:w2_f_6
18013:w2_f_6
18014:w2_f_6
18015:w2_f_6
18016:w2_f_6
18017:w2_f_6
18018:w2_f_6
18019:w2_f_6
18020:w2_f_6
18021:w2_f_6
18022:w2_f_6
18023:w2_f_6
18024:w2_f_6
18025:w2_f_6
18026:w2_f_6
18027:w2_f_6
18028:w2_f_6
18029:w2_f_6
18030:w2_f_6
18031:w2_f_6
18032:w2_f_6
18033:w2_f_6
18034:w2_f_6
18035:w2_f_6
18036:w2_f_6
18037:w2_f_6
18038:w2_f_6
18039:w2_f_6
18040:w2_f_6
18041:w2_f_6
18042:w2_f_6
18043:w2_f_6
18044:w2_f_6
18045:w2_f_6
18046:w2_f_6
18047:w2_f_6
18048:w2_f_6
18049:w2_f_6
18050:w2_f_6
18051:w2_f_6
18052:w2_f_6
18053:w2_f_6
18054:w2_f_6
18055:w2_f_6
18056:w2_f_6
18057:w2_f_6
18058:w2_f_6
18059:w2_f_6

18656:w2_f_6
18657:w2_f_6
18658:w2_f_6
18659:w2_f_6
18660:w2_f_6
18661:w2_f_6
18662:w2_f_6
18663:w2_f_6
18664:w2_f_6
18665:w2_f_6
18666:w2_f_6
18667:w2_f_6
18668:w2_f_6
18669:w2_f_6
18670:w2_f_6
18671:w2_f_6
18672:w2_f_6
18673:w2_f_6
18674:w2_f_6
18675:w2_f_6
18676:w2_f_6
18677:w2_f_6
18678:w2_f_6
18679:w2_f_6
18680:w2_f_6
18681:w2_f_6
18682:w2_f_6
18683:w2_f_6
18684:w2_f_6
18685:w2_f_6
18686:w2_f_6
18687:w2_f_6
18688:w2_f_6
18689:w2_f_6
18690:w2_f_6
18691:w2_f_6
18692:w2_f_6
18693:w2_f_6
18694:w2_f_6
18695:w2_f_6
18696:w2_f_6
18697:w2_f_6
18698:w2_f_6
18699:w2_f_6
18700:w2_f_6
18701:w2_f_6
18702:w2_f_6
18703:w2_f_6
18704:w2_f_6
18705:w2_f_6
18706:w2_f_6
18707:w2_f_6
18708:w2_f_6
18709:w2_f_6
18710:w2_f_6
18711:w2_f_6
18712:w2_f_6
18713:w2_f_6
18714:w2_f_6
18715:w2_f_6
18716:w2_f_6
18717:w2_f_6
18718:w2_f_6
18719:w2_f_6
18720:w2_f_6
18721:w2_f_6
18722:w2_f_6
18723:w2_f_6
18724:w2_f_6
18725:w2_f_6
18726:w2_f_6
18727:w2_f_6
18728:w2_f_6
18729:w2_f_6
18730:w2_f_6
18731:w2_f_6
18732:w2_f_6

19418:w2_f_6
19419:w2_f_6
19420:w2_f_6
19421:w2_f_6
19422:w2_f_6
19423:w2_f_6
19424:w2_f_6
19425:w2_f_6
19426:w2_f_6
19427:w2_f_6
19428:w2_f_6
19429:w2_f_6
19430:w2_f_6
19431:w2_f_6
19432:w2_f_6
19433:w2_f_6
19434:w2_f_6
19435:w2_f_6
19436:w2_f_6
19437:w2_f_6
19438:w2_f_6
19439:w2_f_6
19440:w2_f_6
19441:w2_f_6
19442:w2_f_6
19443:w2_f_6
19444:w2_f_6
19445:w2_f_6
19446:w2_f_6
19447:w2_f_6
19448:w2_f_6
19449:w2_f_6
19450:w2_f_6
19451:w2_f_6
19452:w2_f_6
19453:w2_f_6
19454:w2_f_6
19455:w2_f_6
19456:w2_f_6
19457:w2_f_6
19458:w2_f_6
19459:w2_f_6
19460:w2_f_6
19461:w2_f_6
19462:w2_f_6
19463:w2_f_6
19464:w2_f_6
19465:w2_f_6
19466:w2_f_6
19467:w2_f_6
19468:w2_f_6
19469:w2_f_6
19470:w2_f_6
19471:w2_f_6
19472:w2_f_6
19473:w2_f_6
19474:w2_f_6
19475:w2_f_6
19476:w2_f_6
19477:w2_f_6
19478:w2_f_6
19479:w2_f_6
19480:w2_f_6
19481:w2_f_6
19482:w2_f_6
19483:w2_f_6
19484:w2_f_6
19485:w2_f_6
19486:w2_f_6
19487:w2_f_6
19488:w2_f_6
19489:w2_f_6
19490:w2_f_6
19491:w2_f_6
19492:w2_f_6
19493:w2_f_6
19494:w2_f_6

20168:w2_f_6
20169:w2_f_6
20170:w2_f_6
20171:w2_f_6
20172:w2_f_6
20173:w2_f_6
20174:w2_f_6
20175:w2_f_6
20176:w2_f_6
20177:w2_f_6
20178:w2_f_6
20179:w2_f_6
20180:w2_f_6
20181:w2_f_6
20182:w2_f_6
20183:w2_f_6
20184:w2_f_6
20185:w2_f_6
20186:w2_f_6
20187:w2_f_6
20188:w2_f_6
20189:w2_f_6
20190:w2_f_6
20191:w2_f_6
20192:w2_f_6
20193:w2_f_6
20194:w2_f_6
20195:w2_f_6
20196:w2_f_6
20197:w2_f_6
20198:w2_f_6
20199:w2_f_6
20200:w2_f_6
20201:w2_f_6
20202:w2_f_6
20203:w2_f_6
20204:w2_f_6
20205:w2_f_6
20206:w2_f_6
20207:w2_f_6
20208:w2_f_6
20209:w2_f_6
20210:w2_f_6
20211:w2_f_6
20212:w2_f_6
20213:w2_f_6
20214:w2_f_6
20215:w2_f_6
20216:w2_f_6
20217:w2_f_6
20218:w2_f_6
20219:w2_f_6
20220:w2_f_6
20221:w2_f_6
20222:w2_f_6
20223:w2_f_6
20224:w2_f_6
20225:w2_f_6
20226:w2_f_6
20227:w2_f_6
20228:w2_f_6
20229:w2_f_6
20230:w2_f_6
20231:w2_f_6
20232:w2_f_6
20233:w2_f_6
20234:w2_f_6
20235:w2_f_6
20236:w2_f_6
20237:w2_f_6
20238:w2_f_6
20239:w2_f_6
20240:w2_f_6
20241:w2_f_6
20242:w2_f_6
20243:w2_f_6
20244:w2_f_6

20835:w2_f_6
20836:w2_f_6
20837:w2_f_6
20838:w2_f_6
20839:w2_f_6
20840:w2_f_6
20841:w2_f_6
20842:w2_f_6
20843:w2_f_6
20844:w2_f_6
20845:w2_f_6
20846:w2_f_6
20847:w2_f_6
20848:w2_f_6
20849:w2_f_6
20850:w2_f_6
20851:w2_f_6
20852:w2_f_6
20853:w2_f_6
20854:w2_f_6
20855:w2_f_6
20856:w2_f_6
20857:w2_f_6
20858:w2_f_6
20859:w2_f_6
20860:w2_f_6
20861:w2_f_6
20862:w2_f_6
20863:w2_f_6
20864:w2_f_6
20865:w2_f_6
20866:w2_f_6
20867:w2_f_6
20868:w2_f_6
20869:w2_f_6
20870:w2_f_6
20871:w2_f_6
20872:w2_f_6
20873:w2_f_6
20874:w2_f_6
20875:w2_f_6
20876:w2_f_6
20877:w2_f_6
20878:w2_f_6
20879:w2_f_6
20880:w2_f_6
20881:w2_f_6
20882:w2_f_6
20883:w2_f_6
20884:w2_f_6
20885:w2_f_6
20886:w2_f_6
20887:w2_f_6
20888:w2_f_6
20889:w2_f_6
20890:w2_f_6
20891:w2_f_6
20892:w2_f_6
20893:w2_f_6
20894:w2_f_6
20895:w2_f_6
20896:w2_f_6
20897:w2_f_6
20898:w2_f_6
20899:w2_f_6
20900:w2_f_6
20901:w2_f_6
20902:w2_f_6
20903:w2_f_6
20904:w2_f_6
20905:w2_f_6
20906:w2_f_6
20907:w2_f_6
20908:w2_f_6
20909:w2_f_6
20910:w2_f_6
20911:w2_f_6

In [270]:
in_db = lmdb.open(test_lmdb, map_size=int(1e12))
path = "../../dataset/AND_dataset/AND_dataset/AND_images/"
with in_db.begin(write=True) as in_txn:
    for rows in range(len(test_images)):
        path1 = test_images["w1_f_6"][rows]
        path2 = test_images["w2_f_6"][rows]
        label = test_label["sameWriter"][rows]
        img1 = cv2.imread(str(path+path1), cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(str(path+path2), cv2.IMREAD_GRAYSCALE)
        datum = make_datum(img1,img2,label)
        in_txn.put('{:0>5d}'.format(rows).encode(), datum.SerializeToString())
        print('{:0>5d}'.format(rows) + ':' + str(path1) + ',' + str(path2))
in_db.close()

00000:1102b_num3.png,1102a_num2.png
00001:0117c_num4.png,0117a_num1.png
00002:1157c_num2.png,0618c_num2.png
00003:0468b_num5.png,0468c_num1.png
00004:0992b_num4.png,0488c_num2.png
00005:1003a_num3.png,1003c_num1.png
00006:1011a_num1.png,1011b_num4.png
00007:0343b_num3.png,0343a_num3.png
00008:1503c_num2.png,1503a_num1.png
00009:0380b_num1.png,0712b_num1.png
00010:0219b_num3.png,0219c_num1.png
00011:0982a_num1.png,0254b_num4.png
00012:0810b_num1.png,0810b_num7.png
00013:0271b_num4.png,0271a_num2.png
00014:1194b_num1.png,1194b_num4.png
00015:0621a_num3.png,0973b_num1.png
00016:0282b_num1.png,0122c_num1.png
00017:0519a_num3.png,1265c_num2.png
00018:0721c_num6.png,0721c_num4.png
00019:0931c_num1.png,0931c_num5.png
00020:0069a_num3.png,0069c_num4.png
00021:0018a_num3.png,0569a_num4.png
00022:0265c_num4.png,0165b_num1.png
00023:0236a_num2.png,0683c_num2.png
00024:0107a_num4.png,1008b_num2.png
00025:0955c_num5.png,0955c_num3.png
00026:0485a_num3.png,1368b_num1.png
00027:1554a_num4.png,1537c_n

00305:0672a_num2.png,0672c_num2.png
00306:1097b_num2.png,1097c_num2.png
00307:1106a_num1.png,0227c_num2.png
00308:0219b_num2.png,0219c_num3.png
00309:0535b_num7.png,0504b_num1.png
00310:1446c_num2.png,1446c_num1.png
00311:0239a_num1.png,0557c_num3.png
00312:0505aa_num2.png,0505c_num2.png
00313:0335c_num1.png,0335b_num4.png
00314:0180a_num1.png,0339b_num4.png
00315:1331b_num5.png,1331b_num4.png
00316:0634b_num1.png,0634c_num3.png
00317:0525a_num3.png,0525c_num3.png
00318:0230a_num4.png,0230c_num1.png
00319:0609a_num2.png,0915b_num4.png
00320:0002a_num2.png,0179c_num1.png
00321:0132c_num4.png,0640bb_num2.png
00322:0458a_num1.png,0437b_num5.png
00323:1546a_num5.png,1112a_num7.png
00324:1323b_num3.png,0855b_num2.png
00325:1527c_num1.png,1527b_num1.png
00326:0094a_num2.png,0491a_num4.png
00327:0827b_num6.png,1093b_num2.png
00328:0086b_num3.png,1276a_num1.png
00329:1566b_num4.png,1463c_num1.png
00330:0270c_num4.png,0138b_num3.png
00331:1105c_num1.png,1105b_num1.png
00332:1086b_num1.png,0185a

00613:1437b_num5.png,1437a_num2.png
00614:0341c_num4.png,0341c_num1.png
00615:0820a_num3.png,0820a_num2.png
00616:0391a_num3.png,0942b_num1.png
00617:1049a_num3.png,0991a_num4.png
00618:0126c_num2.png,0734c_num2.png
00619:0269b_num4.png,0501a_num1.png
00620:0384c_num2.png,0479c_num4.png
00621:0221a_num4.png,1409b_num1.png
00622:0284a_num2.png,0284c_num1.png
00623:0203b_num2.png,0203a_num1.png
00624:1183a_num1.png,1183c_num1.png
00625:0866c_num1.png,0848c_num1.png
00626:0663a_num1.png,0663c_num1.png
00627:0215a_num3.png,0215c_num2.png
00628:0437c_num4.png,1246c_num2.png
00629:0650a_num4.png,0091a_num3.png
00630:0146b_num2.png,1511c_num3.png
00631:0604b_num2.png,0955b_num1.png
00632:1185c_num4.png,0199a_num4.png
00633:0425c_num1.png,0428b_num2.png
00634:0177b_num3.png,0177a_num1.png
00635:1042a_num2.png,0998a_num5.png
00636:0675a_num1.png,0675b_num1.png
00637:0386a_num2.png,0386b_num2.png
00638:0410c_num1.png,1433b_num5.png
00639:0918a_num3.png,0918b_num3.png
00640:0281a_num2.png,0339c_n

00936:0862c_num1.png,1330a_num4.png
00937:1270c_num2.png,0141c_num1.png
00938:0169a_num4.png,0075c_num1.png
00939:0540a_num1.png,0540c_num1.png
00940:0817a_num4.png,0354c_num2.png
00941:0291a_num1.png,0705a_num5.png
00942:0324b_num1.png,0324a_num1.png
00943:0606c_num2.png,0606a_num2.png
00944:0594c_num5.png,0594c_num2.png
00945:0935a_num1.png,0935b_num1.png
00946:0177a_num4.png,0598a_num1.png
00947:0393a_num1.png,0393a_num2.png
00948:0798c_num5.png,0798c_num2.png
00949:1082c_num1.png,1082a_num2.png
00950:0354c_num1.png,0354c_num2.png
00951:0222a_num3.png,0222a_num4.png
00952:1048b_num5.png,1048a_num3.png
00953:0060c_num1.png,1276b_num1.png
00954:1178a_num3.png,1178b_num3.png
00955:0901b_num3.png,0901a_num3.png
00956:0047a_num2.png,0047b_num3.png
00957:0516c_num2.png,0739a_num1.png
00958:0038a_num6.png,0654a_num1.png
00959:0984b_num3.png,0984a_num3.png
00960:1291a_num4.png,1291b_num4.png
00961:1292b_num2.png,1292b_num3.png
00962:0194b_num3.png,0194a_num1.png
00963:1148c_num5.png,0384b_n

01169:0192c_num3.png,1104b_num1.png
01170:1230a_num2.png,0357b_num4.png
01171:1030b_num4.png,0697b_num4.png
01172:0587a_num2.png,0587b_num3.png
01173:1085c_num5.png,0868aa_num1.png
01174:0413b_num1.png,0413a_num2.png
01175:1428c_num2.png,1428a_num1.png
01176:1006a_num3.png,1006b_num4.png
01177:1531b_num1.png,1002b_num5.png
01178:1196c_num5.png,0102c_num4.png
01179:1425a_num1.png,0906b_num1.png
01180:0256b_num1.png,0304a_num4.png
01181:1235c_num1.png,1235b_num2.png
01182:0323a_num1.png,0323c_num1.png
01183:0258c_num2.png,0258c_num1.png
01184:1050b_num1.png,1050a_num4.png
01185:0211a_num1.png,0765b_num4.png
01186:1427c_num3.png,1427b_num1.png
01187:0900b_num1.png,0677b_num1.png
01188:0950a_num3.png,0950c_num1.png
01189:0019c_num1.png,0465a_num2.png
01190:0934b_num1.png,0621b_num2.png
01191:0227c_num5.png,1183b_num6.png
01192:0941a_num5.png,1257a_num2.png
01193:0299a_num3.png,0450a_num5.png
01194:0848b_num4.png,0794a_num4.png
01195:0892c_num1.png,0892a_num2.png
01196:0985c_num5.png,0985c_

...

03889:0922b_num3.png,0922b_num1.png
03890:0871b_num2.png,0135b_num2.png
03891:0667c_num2.png,0667a_num2.png
03892:0678b_num2.png,0123c_num1.png
03893:0701b_num3.png,0701c_num3.png
03894:0307a_num1.png,0307b_num1.png
03895:0484c_num1.png,0484c_num3.png
03896:0471c_num1.png,1067a_num1.png
03897:0168b_num1.png,0168b_num2.png
03898:1438a_num4.png,1478c_num1.png
03899:1158b_num4.png,1111c_num4.png
03900:0355bb_num1.png,0082c_num3.png
03901:0502a_num3.png,0502b_num2.png
03902:0766c_num3.png,0346c_num1.png
03903:0384c_num6.png,0384b_num1.png
03904:0662a_num1.png,0770a_num2.png
03905:1178c_num4.png,1178b_num1.png
03906:0417a_num2.png,0417c_num2.png
03907:0687a_num6.png,0687b_num5.png
03908:1170c_num2.png,1170a_num2.png
03909:0181b_num5.png,1328b_num1.png
03910:0583a_num1.png,0583b_num2.png
03911:0378b_num7.png,1323b_num7.png
03912:0113c_num2.png,0691b_num2.png
03913:0052b_num4.png,1237a_num4.png
03914:0299a_num3.png,0299c_num2.png
03915:1444c_num1.png,0721c_num3.png
03916:1374a_num2.png,1374c_

04237:0767c_num1.png,0310a_num4.png
04238:0556b_num6.png,0556c_num4.png
04239:0990a_num3.png,1058b_num1.png
04240:1215c_num1.png,0280c_num2.png
04241:0815a_num3.png,1135b_num3.png
04242:0327a_num4.png,0181c_num5.png
04243:0393c_num1.png,0393b_num2.png
04244:0120b_num1.png,0261a_num3.png
04245:0356b_num4.png,0356c_num3.png
04246:0734a_num2.png,0169a_num2.png
04247:0705c_num3.png,0705b_num3.png
04248:0343a_num1.png,0691c_num5.png
04249:1260c_num3.png,1260b_num3.png
04250:0758b_num1.png,0758c_num2.png
04251:0959c_num5.png,0959b_num3.png
04252:1527a_num2.png,1527a_num1.png
04253:0688c_num1.png,0688a_num3.png
04254:0800c_num2.png,0800c_num3.png
04255:0742b_num1.png,0966b_num3.png
04256:0522c_num2.png,0522b_num1.png
04257:1294a_num1.png,0905a_num3.png
04258:1335a_num3.png,0442b_num1.png
04259:0630b_num1.png,0630b_num2.png
04260:1437c_num2.png,1437b_num6.png
04261:0964c_num3.png,0261a_num2.png
04262:1112c_num6.png,0089a_num2.png
04263:0652a_num4.png,0207c_num3.png
04264:0738b_num1.png,1524a_n

04504:1191a_num3.png,0091b_num1.png
04505:1199b_num4.png,0297b_num4.png
04506:0644b_num2.png,0990a_num4.png
04507:1429c_num2.png,1429b_num4.png
04508:1231b_num1.png,0325a_num3.png
04509:1266c_num6.png,0726a_num1.png
04510:0255c_num1.png,0976a_num1.png
04511:0977b_num4.png,0977c_num2.png
04512:1404aa_num1.png,0930a_num2.png
04513:1135c_num1.png,1135b_num1.png
04514:0618b_num2.png,1477b_num1.png
04515:0624b_num1.png,0411c_num5.png
04516:0267c_num4.png,0678a_num1.png
04517:1324c_num1.png,1324b_num1.png
04518:1167b_num4.png,1167a_num2.png
04519:0695a_num2.png,0573b_num1.png
04520:0524aa_num1.png,1237a_num3.png
04521:0094c_num2.png,0094a_num6.png
04522:0192c_num2.png,0510b_num6.png
04523:0880b_num1.png,0942a_num2.png
04524:0471b_num3.png,0471a_num3.png
04525:1188c_num2.png,0685c_num1.png
04526:0991c_num5.png,0928c_num1.png
04527:1170c_num3.png,0368c_num1.png
04528:0242c_num3.png,0132b_num1.png
04529:0734a_num2.png,0734b_num2.png
04530:1231b_num1.png,1231c_num3.png
04531:1258a_num1.png,0525c

04820:1356b_num2.png,0907a_num2.png
04821:1234c_num4.png,1287c_num2.png
04822:1370a_num2.png,1370a_num3.png
04823:0084c_num2.png,0084cc_num1.png
04824:0059a_num2.png,0059a_num3.png
04825:0939b_num3.png,0396b_num1.png
04826:1114aa_num1.png,1114b_num1.png
04827:0735b_num1.png,0735a_num3.png
04828:0135c_num2.png,0135b_num2.png
04829:0451c_num1.png,0142b_num3.png
04830:0328c_num5.png,0328aa_num1.png
04831:0678a_num1.png,0273c_num4.png
04832:0677b_num3.png,0132c_num4.png
04833:0243c_num2.png,0370b_num1.png
04834:0848a_num1.png,0589a_num1.png
04835:1445a_num4.png,1445a_num1.png
04836:0985a_num1.png,0629b_num3.png
04837:0912a_num1.png,0474a_num2.png
04838:0813b_num5.png,0768a_num1.png
04839:1004c_num1.png,1004b_num2.png
04840:1235c_num5.png,1235c_num2.png
04841:1343a_num4.png,1343a_num5.png
04842:0262b_num1.png,0528b_num2.png
04843:0102b_num2.png,0102a_num1.png
04844:0311b_num1.png,0256c_num4.png
04845:0546b_num3.png,0255a_num1.png
04846:0065b_num3.png,1097c_num4.png
04847:1165c_num2.png,0005

05147:0456b_num1.png,0456a_num2.png
05148:1203b_num3.png,1406a_num2.png
05149:1496c_num2.png,0002b_num5.png
05150:1273b_num1.png,1273a_num4.png
05151:1050b_num2.png,1050c_num6.png
05152:0300a_num1.png,0300b_num3.png
05153:1105b_num2.png,1375a_num1.png
05154:0137c_num3.png,0137cc_num2.png
05155:1200c_num1.png,1200a_num2.png
05156:1231c_num5.png,0307b_num4.png
05157:0481b_num5.png,0998b_num4.png
05158:0779c_num2.png,0842a_num2.png
05159:1539b_num2.png,1539c_num2.png
05160:1279c_num6.png,1115b_num1.png
05161:1489c_num3.png,1489a_num3.png
05162:0756b_num2.png,1374b_num1.png
05163:1011b_num2.png,1011b_num6.png
05164:1109c_num1.png,1058c_num3.png
05165:0914c_num4.png,1306c_num1.png
05166:0810b_num7.png,0810a_num5.png
05167:0939a_num3.png,0939b_num3.png
05168:0172aa_num1.png,0880c_num2.png
05169:0797a_num1.png,1564aa_num1.png
05170:0337c_num5.png,0337b_num4.png
05171:0064a_num3.png,0064b_num1.png
05172:0562a_num4.png,0562b_num2.png
05173:1170c_num3.png,1170b_num1.png
05174:0703c_num2.png,0468

In [8]:
model = caffe.Net("siamese.prototxt","./siamese_iter_1722.caffemodel",caffe.TEST)

In [33]:
model.blobs

OrderedDict([('pair_data', <caffe._caffe.Blob at 0x7ff8b69e7f50>),
             ('sim', <caffe._caffe.Blob at 0x7ff8b69e7660>),
             ('data', <caffe._caffe.Blob at 0x7ff8b69e7768>),
             ('data_p', <caffe._caffe.Blob at 0x7ff8b69e74a8>),
             ('conv1', <caffe._caffe.Blob at 0x7ff8b69e7d98>),
             ('pool1', <caffe._caffe.Blob at 0x7ff8b69e7df0>),
             ('conv2', <caffe._caffe.Blob at 0x7ff8b69e7e48>),
             ('pool2', <caffe._caffe.Blob at 0x7ff8b69e7ea0>),
             ('ip1', <caffe._caffe.Blob at 0x7ff8b69e7608>),
             ('ip2', <caffe._caffe.Blob at 0x7ff8b69e7a80>),
             ('feat', <caffe._caffe.Blob at 0x7ff8b69e79d0>),
             ('conv1_p', <caffe._caffe.Blob at 0x7ff8b69e7fa8>),
             ('pool1_p', <caffe._caffe.Blob at 0x7ff8b69e7ad8>),
             ('conv2_p', <caffe._caffe.Blob at 0x7ff8b69e76b8>),
             ('pool2_p', <caffe._caffe.Blob at 0x7ff8b69e7a28>),
             ('ip1_p', <caffe._caffe.Blob at 0x7ff

In [84]:
img1 = cv2.imread("../../dataset/AND_dataset/AND_dataset/AND_images/1013a_num2.png",cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread("../../dataset/AND_dataset/AND_dataset/AND_images/1225a_num3.png",cv2.IMREAD_GRAYSCALE)

In [85]:
model.blobs['pair_data'].reshape(1,2,64,64)

In [86]:
model.blobs['pair_data'].data[...] = np.array([img1,img2])

In [87]:
out = model.forward()

In [88]:
out

{'loss': array(0.13721955, dtype=float32)}

,w1_f_6,w2_f_6,sameWriter
0,1552a_num1.png,0737a_num2.png,0
1,0145a_num4.png,1379a_num2.png,0
2,0587a_num1.png,0847b_num3.png,0
3,0895a_num2.png,1411a_num3.png,0
4,0450b_num3.png,0450c_num2.png,1
5,0521c_num2.png,0449a_num2.png,0
6,0454b_num1.png,1561c_num1.png,0
7,1386a_num3.png,0613b_num1.png,0
8,1403a_num1.png,0753c_num4.png,0
9,0511b_num3.png,0366c_num2.png,0


In [15]:
from caffe_weight_converter import caffe_weight_converter

In [27]:
caffe_weight_converter.convert_caffemodel_to_keras(output_filename="caffe2KerasMainModel",
                                                  prototxt_filename="siamese.prototxt",
                                                  caffemodel_filename='siamese_iter_1722.caffemodel',
                                                  include_layers_without_weights=False,
                                                  verbose=True)

Skipped layer 'pair_data' of type 'Data' because it doesn't have any weights
Skipped layer 'slice_pair' of type 'Slice' because it doesn't have any weights
Converted weights for layer 'conv1' of type 'Convolution'
Skipped layer 'pool1' of type 'Pooling' because it doesn't have any weights
Converted weights for layer 'conv2' of type 'Convolution'
Skipped layer 'pool2' of type 'Pooling' because it doesn't have any weights
Converted weights for layer 'ip1' of type 'InnerProduct'
Skipped layer 'relu1' of type 'ReLU' because it doesn't have any weights
Converted weights for layer 'ip2' of type 'InnerProduct'
Converted weights for layer 'feat' of type 'InnerProduct'
Converted weights for layer 'conv1_p' of type 'Convolution'
Skipped layer 'pool1_p' of type 'Pooling' because it doesn't have any weights
Converted weights for layer 'conv2_p' of type 'Convolution'
Skipped layer 'pool2_p' of type 'Pooling' because it doesn't have any weights
Converted weights for layer 'ip1_p' of type 'InnerProdu

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [26]:
model = load_weights('caffe2KerasMainModel.h5')

NameError: name 'load_weights' is not defined